In [2]:
import random
import pandas as pd
import numpy as np

# Matriz
E = [(1,2),(1,9),(1,14),(2,3),(2,5),(3,4),(3,5),(3,6),(3,7),(4,6),(5,7),(5,9),(5,10),(6,7),(6,8),(7,8),(7,11),(8,11),(9,10),(9,13),(9,14),(10,11),(10,12),(12,16),(13,15),(13,16),(14,15),(15,16)]
df = np.zeros((16, 16))
for row, col in E:
    df[row-1, col-1] = 1
    df[col-1, row-1] = 1  # Añadir conexión bidireccional
df = pd.DataFrame(df, columns=range(1, 17), index=range(1, 17))

# Parámetros del algoritmo genético
population_size = 50
num_genes = 16
max_generations = 70
crossover_probability = 0.8
mutation_probability = 0.1
elitism = True #Criterio de reemplazo: Elitismo. Se mantiene al mejor individuo de la generación anterior en la nueva generación.

class Agent:
    def __init__(self, df, num_genes=16):
        self.df = df
        self.num_genes = num_genes
        self.chromosome = self.random_initialization()
        self.fitness = self.calc_fitness()

    def random_initialization(self):
        #Criterio de inicialización: Genera un cromosoma aleatorio y asegura factibilidad.
        chromosome = [random.choice([0, 1]) for _ in range(self.num_genes)]
        while not self.check_factibility(chromosome):
            chromosome = [random.choice([0, 1]) for _ in range(self.num_genes)]
        return chromosome

    def check_factibility(self, chromosome):
        #Criterio de infactibilidad: Verifica que el cromosoma tenga la misma cantidad de 0s y 1s.
        return sum(chromosome) == self.num_genes // 2

    def create_groups(self):
        self.group1 = [i+1 for i in range(self.num_genes) if self.chromosome[i] == 1]
        self.group2 = [i+1 for i in range(self.num_genes) if self.chromosome[i] == 0]

    def calc_connections(self, group1, group2):
        inter_group_edges = [edge for edge in E if (edge[0] in group1 and edge[1] in group2) or (edge[0] in group2 and edge[1] in group1)]
        return len(inter_group_edges)

    def calc_fitness(self):
        #Función fitness: Calcula el número de conexiones entre los dos grupos y lo minimiza.
        self.create_groups()
        self.connections = self.calc_connections(self.group1, self.group2)
        return -self.connections  # Se minimiza el número de conexiones, así que fitness es negativo.

    def mutate(self):
        #Mutación: Realiza una mutación en un gen del cromosoma.
        idx = random.randint(0, self.num_genes - 1)
        self.chromosome[idx] = 1 - self.chromosome[idx]  # Cambio 0 a 1 o 1 a 0
        while not self.check_factibility(self.chromosome):
            idx = random.randint(0, self.num_genes - 1)
            self.chromosome[idx] = 1 - self.chromosome[idx]
        self.fitness = self.calc_fitness()

    def crossover(self, partner):
        #Cruce: Realiza un cruce de un punto entre dos cromosomas.
        if random.random() < crossover_probability:
            crossover_point = random.randint(1, self.num_genes - 2)
            child1_chromosome = self.chromosome[:crossover_point] + partner.chromosome[crossover_point:]
            child2_chromosome = partner.chromosome[:crossover_point] + self.chromosome[crossover_point:]

            child1 = Agent(self.df, self.num_genes)
            child2 = Agent(self.df, self.num_genes)
            child1.chromosome = child1_chromosome
            child2.chromosome = child2_chromosome

            if child1.check_factibility(child1_chromosome):
                child1.fitness = child1.calc_fitness()
            if child2.check_factibility(child2_chromosome):
                child2.fitness = child2.calc_fitness()

            return child1, child2
        else:
            return self, partner

def selection(population):
    #Criterio de selección: Selección por torneo.
    selected = []
    for _ in range(len(population)):
        ind1 = random.choice(population)
        ind2 = random.choice(population)
        selected.append(ind1 if ind1.fitness > ind2.fitness else ind2)
    return selected

def replacement(old_population, new_population):
    #Criterio de reemplazo: Elitismo.
    if elitism:
        combined = old_population + new_population
        combined.sort(key=lambda x: x.fitness, reverse=True)
        return combined[:population_size]
    else:
        return new_population

# Inicializar población
population = [Agent(df, num_genes) for _ in range(population_size)]

# Algoritmo Genético
for generation in range(max_generations):
    new_population = []

    selected = selection(population)

    # Cruce
    for i in range(0, population_size, 2):
        parent1 = selected[i]
        parent2 = selected[i + 1]
        child1, child2 = parent1.crossover(parent2)
        new_population.extend([child1, child2])

    # Mutación
    for individual in new_population:
        if random.random() < mutation_probability:
            individual.mutate()

    # Reemplazo
    population = replacement(population, new_population)

    # Criterio de paro: Si se llega al número máximo de generaciones
    best_fitness = max([agent.fitness for agent in population])
    print(f"Generation {generation}: Best Fitness: {best_fitness}")

# Resultado final
best_agent = max(population, key=lambda x: x.fitness)
print("Best partition found:")
print("Group 1:", best_agent.group1)
print("Group 2:", best_agent.group2)
print("Minimum connections:", -best_agent.fitness)


Generation 0: Best Fitness: -8
Generation 1: Best Fitness: -8
Generation 2: Best Fitness: -8
Generation 3: Best Fitness: -8
Generation 4: Best Fitness: -6
Generation 5: Best Fitness: -6
Generation 6: Best Fitness: -6
Generation 7: Best Fitness: -6
Generation 8: Best Fitness: -6
Generation 9: Best Fitness: -6
Generation 10: Best Fitness: -6
Generation 11: Best Fitness: -6
Generation 12: Best Fitness: -6
Generation 13: Best Fitness: -6
Generation 14: Best Fitness: -6
Generation 15: Best Fitness: -6
Generation 16: Best Fitness: -4
Generation 17: Best Fitness: -4
Generation 18: Best Fitness: -4
Generation 19: Best Fitness: -4
Generation 20: Best Fitness: -4
Generation 21: Best Fitness: -4
Generation 22: Best Fitness: -4
Generation 23: Best Fitness: -4
Generation 24: Best Fitness: -4
Generation 25: Best Fitness: -4
Generation 26: Best Fitness: -4
Generation 27: Best Fitness: -4
Generation 28: Best Fitness: -4
Generation 29: Best Fitness: -4
Generation 30: Best Fitness: -4
Generation 31: Bes